In [7]:
import PySimpleGUI as sg
import lightkurve as lk
import numpy as np
import pandas as pd

####### THEME #######
sg.theme('DarkGray1')
#####################

#LAYOUT
layout = [
    [sg.Radio('Target', 'tgt' , key='-TARGET_RADIO-', default=True), sg.Input('', key='-TARGET_INPUT-', size=(45, 0), pad=(51, 5))],
    [sg.Radio('List of Targets', 'tgt', key='-TARGET_LIST_RADIO-'), sg.Input(key='-TARGET_FILE-'), sg.FileBrowse(enable_events=True, size=(12, 0))],
    [sg.HSeparator()],
    [sg.Radio('Sector', 'sct', key='-SEC_RADIO-', default=True), sg.Input('', key='-SEC_INPUT-', size=(10, 0), pad=(51, 5))],
    [sg.Radio('List of Sectors', 'sct', key='-SEC_LIST_RADIO-'), sg.Input('', key='-SEC_LIST_INPUT-', pad=(5, 5))],
    [sg.Radio('All Sectors (merged)', 'sct', key='-ALL_SEC_RADIO-')],
    [sg.HSeparator()],
    [sg.Push(), sg.Button('DOWNLOAD', key='-DOWNLOAD_BUTTON-', size=(12, 2)), sg.Push()],
    [sg.ProgressBar(100, size=(500, 15), bar_color=('green', 'lightgrey'))]
    
]

# FUNCTIONALITIES
def single_target(tic, sector):
    search = lk.search_lightcurve(target=f'TIC {tic}', sector=sector, author='SPOC', exptime=120)
    lc = search.download().normalize().remove_outliers(sigma=3.5)
    time = lc.time.value
    flux = lc.flux.value
    df = pd.DataFrame(np.column_stack([time,flux]), columns=['Time', 'Flux'] )
    return df
    


# JANELA
window = sg.Window('TL Download Project', layout=layout, size=(550, 300))


while True:
    event, value = window.read()
    
    if event == sg.WIN_CLOSED:
        break
        
    if event == '-DOWNLOAD_BUTTON-':
        ### Download the specified sector of the specified target ###
        if value['-TARGET_RADIO-'] == True and value['-SEC_RADIO-'] == True:
            tic = value['-TARGET_INPUT-']
            sector = value['-SEC_INPUT-']
            data = single_target(tic, sector)
            data.to_csv(f'{tic.zfill(10)}_{str(sector).zfill(2)}.dat', sep=' ', index=None, header=None)
            
            sg.popup('File saved successfully!')
            
        
window.close()